In [1]:
import wikipediaapi
from IPython.display import Image, display
import ipywidgets as widgets
from bs4 import BeautifulSoup
import re
import os
import requests
import pyttsx3

In [2]:
def test_voices():
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    print({i : voice.name for i, voice in enumerate(voices)})
    for i, voice in enumerate(voices):
        print(i, voice.name)
        if '(' not in voice.name:
            engine.setProperty('voice', voice.id)
            engine.say(f'I am now Testing the voice of {voice.name}')
    engine.runAndWait()

In [3]:
def get_data(request):
    wiki_page = wikipediaapi.Wikipedia(user_agent='Schaolar', language = 'en')
    page = wiki_page.page(request)

    return page

def get_image(request, idx = 1):
    url = f"https://www.google.com/search?client=firefox-b-1-d&sca_esv=f6cb87a6dc41573d&q={request}&udm=2&fbs=AIIjpHxU7SXXniUZfeShr2fp4giZ1Y6MJ25_tmWITc7uy4KIeiAkWG4OlBE2zyCTMjPbGmMU8EWskMk2JSE__efdUJ3x-dd8PyEzi5Y9BtPQcYyUv-qqGBKcZRAt6t3qPZ3-iSZ6GtBRrUomyC2rUgS4TDygQfIaiWCtBi0gw2eDDygE9ZsO235rUXAUX4XMOSF0AZ0oNb07blcuzshZNivp14ZK7ZppxA&sa=X&ved=2ahUKEwiw09-G8tqOAxURk4kEHVdoBBcQtKgLKAF6BAgmEAE&biw=1252&bih=1014&dpr=1" # Google Images URL

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    images = soup.find_all('img') 
    image_data = requests.get(images[idx]['src'])

    image_widget1 = widgets.Image(
        value=image_data.content,
        format='jpg',
        layout=widgets.Layout(object_fit = 'contain'))

    return image_widget1
    

In [4]:
valid_schaolars = ['robot', 'cat', 'hamster', 'angrybot', 'infinity', 'creepy']
schaolar_info = {'robot' : (200, 64),
                 'cat' : (200, 16), 
                 'hamster' : (100, 1), 
                 'angrybot' : (200, 100), 
                 'infinity' : (200, 47),
                 'creepy' : (200, 102)}

def get_schaolar(schaolar):
    if schaolar == 'user_select':
        print(f"Schaolars: {valid_schaolars}")

    while schaolar not in valid_schaolars:
        schaolar = input('Select Schaolar (see list)')
        if schaolar not in valid_schaolars:
            print(f'Schaolar not in list : {valid_schaolars}')

    return schaolar
            
def get_schaolar_im(schaolar = 'robot'):
    with open(f'schaolars/{schaolar}_schaolar.gif', 'rb') as f:
        return widgets.Image(value = f.read(), format = 'gif',
                             layout=widgets.Layout(object_fit='contain'))

In [5]:
def ask_schaolar(schaolar = 'robot'):
    schaolar = get_schaolar(schaolar)

    request = input('What topic do you want to learn about? ')
    text = get_data(request).summary

    engine = pyttsx3.init()

    engine.setProperty('rate', schaolar_info[schaolar][0])
    engine.setProperty('volume',1.0)

    # VOICE
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[schaolar_info[schaolar][1]].id) 

    schaolar_im = get_schaolar_im(schaolar)
    topic_im = get_image(request, 1)
    topic_im2 = get_image(request, 2)

    box = widgets.HBox([schaolar_im, topic_im, topic_im2])
    display(box)

    engine.say(text)
    engine.runAndWait()

In [ ]:
ask_schaolar('user_select')

Schaolars: ['robot', 'cat', 'hamster', 'angrybot', 'infinity', 'creepy']
